In [1]:
!pip install PyPDF2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 4.9 MB/s eta 0:00:00


In [2]:
!pip install python-dotenv

In [3]:
!pip install langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 811.8/811.8 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 18.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.4/239.4 kB 17.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.7/55.7 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.4/55.4 kB 7.8 MB/s eta 0:00:00


In [4]:
!pip install langchain-google-genai

In [15]:
!pip install faiss-cpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 53.5 MB/s eta 0:00:00


In [10]:
from google.colab import userdata

In [11]:
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)

In [12]:
GOOGLE_API_KEY = ""

In [16]:
from PyPDF2 import PdfReader
from langchain.text_splitter import RecursiveCharacterTextSplitter
import os
from langchain_google_genai import GoogleGenerativeAIEmbeddings
import google.generativeai as genai
from langchain.vectorstores import FAISS
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.chains.question_answering import load_qa_chain
from langchain.prompts import PromptTemplate
from dotenv import load_dotenv

# Set your Google API key here (obtain one from https://console.cloud.google.com/)
os.environ["GOOGLE_API_KEY"] = ""

def get_pdf_text(pdf_path):
    text = ""
    with open(pdf_path, "rb") as f:
        pdf_reader = PdfReader(f)
        for page in pdf_reader.pages:
            text += page.extract_text()
    return text

def get_text_chunks(text):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=10000, chunk_overlap=1000)
    chunks = text_splitter.split_text(text)
    return chunks

def get_vector_store(text_chunks):
    embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001",google_api_key=GOOGLE_API_KEY)
    vector_store = FAISS.from_texts(text_chunks, embedding=embeddings)
    vector_store.save_local("faiss_index")

def get_conversational_chain():
    prompt_template = """
        You are a doctor reviewing a medical report for a patient.
        Provide answers to patient's questions and suggest diet plans based on the report.
        Mention possible diseases if a healthy diet is not followed, based on the report's parameters.

        Context: {context}
        Question: {question}

        Answer:
    """

    model = ChatGoogleGenerativeAI(model="gemini-pro", temperature=0.3)
    prompt = PromptTemplate(template=prompt_template, input_variables=["context", "question"])
    chain = load_qa_chain(model, chain_type="stuff", prompt=prompt)
    return chain

def process_and_respond(pdf_path):
    # Process PDF text
    raw_text = get_pdf_text(pdf_path)
    text_chunks = get_text_chunks(raw_text)
    get_vector_store(text_chunks)

    # Initiate conversation without requiring a question
    context = raw_text  # Provide entire PDF text as context
    question = ""  # No initial question required
    chain = get_conversational_chain()
    response = chain({"input_documents": [], "context": context, "question": question})

    return response["output_text"]


In [17]:
# Example usage
if __name__ == "__main__":
    pdf_path = "/content/report.pdf"  # Replace with your PDF path
    response = process_and_respond(pdf_path)
    print("**Insights from the PDF:**")
    print(response)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


**Insights from the PDF:**
**Question:** I'm concerned about my cholesterol levels. What can I do to lower them?

**Answer:** Based on your medical report, your cholesterol levels are slightly elevated. To lower your cholesterol, you should follow a healthy diet that is low in saturated and trans fats, and high in soluble fiber. You should also get regular exercise and maintain a healthy weight.

**Diet Plan:**

* **Breakfast:** Oatmeal with berries and nuts
* **Lunch:** Salad with grilled chicken or fish
* **Dinner:** Salmon with roasted vegetables
* **Snacks:** Fruits, vegetables, yogurt, or nuts

**Possible Diseases if a Healthy Diet is Not Followed:**

* **Heart disease:** High cholesterol is a major risk factor for heart disease, including heart attack and stroke.
* **Stroke:** High cholesterol can also lead to stroke, which is a sudden loss of blood flow to the brain.
* **Peripheral artery disease:** High cholesterol can also cause peripheral artery disease, which is a narrowing 